In [1]:
from climate.core import EPW

In [2]:
a = EPW("./tests/weatherfile.epw")
a.read()

ValueError: invalid literal for int() with base 10: '0.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00.00

In [3]:
print("Hello Evie!")

Hello Evie!
